# Tutorial data

In [ ]:
# import pandas as pd

# data_path = "data/train/day_0_0.001_train.csv"

# df = pd.read_csv(data_path, header=None)
# df.head()

In [ ]:
# example = df.loc[1, 0]
# example

In [ ]:
# features = example.split('\t')
# len(features)
# features

# Read s3 orc data

In [ ]:
# column_names = []
# with open(f'./schema/column_name_mobivista.txt', 'r') as f:
#     for line in f:
#         column_names.append(line.split(' ')[1].strip())
# print(column_names)

In [ ]:
# import metaspore as ms

# # train_dataset_path = ROOT_DIR + '/data/train/day_0_0.001_train.csv'

# file_base_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'
# num_files = 10
# file_names = [f'part-{str(i).zfill(5)}-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc' for i in range(num_files)]
# train_dataset_path = [file_base_path + file_name for file_name in file_names]

# # train_dataset_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/part-00000-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc'
# # train_dataset_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'

# spark_confs = {
#     'spark.eventLog.enabled':'true',
#     'spark.executor.memory': '10g',
#     'spark.driver.memory': '10g',
#     "spark.driver.maxResultSize": "10g",
#     "spark.sql.files.ignoreCorruptFiles": "true"
# }

# spark_session = ms.spark.get_session(local=True,
#                                      batch_size=100,
#                                      worker_count=2,
#                                      server_count=2,
#                                      log_level='WARN',
#                                      spark_confs=spark_confs)

# train_dataset = ms.input.read_s3_csv(spark_session, 
#                                      train_dataset_path, 
#                                      format='orc',
#                                      shuffle=False,
#                                      delimiter='\t', 
#                                      multivalue_delimiter="\001", 
#                                      column_names=column_names,
#                                      multivalue_column_names=column_names[:-1])

# # train_dataset = spark_session.read.orc(train_dataset_path)

# # train_dataset.printSchema()

In [ ]:
# import pandas as pd
# pd.set_option("display.max_columns", None)

# df = train_dataset.toPandas()
# df.head()

In [ ]:
# num_positive = df[df['label']=='1'].shape[0]
# num_negative = df[df['label']=='0'].shape[0]
# print(f"num_positive: {num_positive}, num_negative: {num_negative}, ratio: {num_positive/num_negative*100}")

In [ ]:
# columns = df.columns.tolist()
# print(columns)


In [ ]:
# total_hash = 0
# feature_hash = {}
# feature_value = {}

# for i, row in df.iterrows():
#     for column in columns[:-1]:
#         if column in feature_value:
#             feature_value[column].append(row[column].shape[0])
#         else:
#             feature_value[column] = [row[column].shape[0]]
        
#         for item in row[column]:
#             if '\003' in item:
#                 hash, weight = item.split('\003')
#             else:
#                 hash, weight = '', 0
#             if column in feature_hash:
#                 feature_hash[column].add(hash)
#             else:
#                 feature_hash[column] = set(hash)

# df_feature = pd.DataFrame(feature_value)
# df_feature.describe()

In [ ]:
# feature_hash_count = dict([(k, [len(v)]) for k, v in feature_hash.items()])
# df_hash = pd.DataFrame(feature_hash_count)
# df_hash

In [ ]:
# total_hash = df_hash.sum(axis=1).values[0]
# print(f"Number of total hashes: {total_hash}")

In [ ]:
columns = []
with open(f'./schema/column_name_mobivista.txt', 'r') as f:
    for line in f:
        columns.append(line.split(' ')[1].strip())
print(columns)

In [ ]:
import metaspore as ms

# Assuming column_names and columns variables are already defined
file_base_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'
num_files = 10
file_names = [f'part-{str(i).zfill(5)}-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc' for i in range(num_files)]
train_dataset_path = [file_base_path + file_name for file_name in file_names]

spark_confs = {
    'spark.eventLog.enabled': 'true',
    'spark.executor.memory': '10g',
    'spark.driver.memory': '10g',
    "spark.driver.maxResultSize": "10g",
    "spark.sql.files.ignoreCorruptFiles": "true"
}

spark_session = ms.spark.get_session(local=True,
                                     batch_size=100,
                                     worker_count=2,
                                     server_count=2,
                                     log_level='WARN',
                                     spark_confs=spark_confs)

train_dataset = ms.input.read_s3_csv(spark_session, 
                                     train_dataset_path, 
                                     format='orc',
                                     shuffle=False,
                                     delimiter='\t', 
                                     multivalue_delimiter="\001", 
                                     column_names=column_names,
                                     multivalue_column_names=column_names[:-1])


In [ ]:

from pyspark.sql.functions import udf, col, sum as spark_sum
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField

# UDF to calculate feature values and hash count
def process_row(column_values):
    feature_values = []
    feature_hashes = set()
    for value in column_values:
        if '\003' in value:
            hash_val, weight = value.split('\003')
        else:
            hash_val, weight = '', 0
        feature_values.append(len(value))
        feature_hashes.add(hash_val)
    return (feature_values, len(feature_hashes))

schema = StructType([
    StructField("feature_values", ArrayType(IntegerType()), True),
    StructField("hash_count", IntegerType(), True)
])

process_row_udf = udf(process_row, schema)

# Applying UDF on each column
for column in columns[:-1]:
    train_dataset = train_dataset.withColumn(f'{column}_processed', process_row_udf(col(column)))

# Extracting the results
result_columns = [f'{column}_processed' for column in columns[:-1]]
processed_df = train_dataset.select(*result_columns)

# Aggregating the results
aggregated_df = processed_df.agg(
    *[spark_sum(col(f'{column}_processed.hash_count')).alias(f'{column}_hash_count') for column in columns[:-1]]
)

# Show results
aggregated_df.show()


In [ ]:

# Calculate total hashes
total_hash = aggregated_df.select(spark_sum(*[col(f'{column}_hash_count') for column in columns[:-1]])).collect()[0][0]
print(f"Number of total hashes: {total_hash}")
